In [1]:
import os
from dotenv import load_dotenv

load_dotenv()
os.chdir(os.getenv("PROJECT_ROOT"))

import numpy as np

## files_helper

In [2]:
from patrec.utils.files_helper import Format1_loader

# load data  
dt = Format1_loader(filepath='data/Synthetic_data/17_06_samples/exp/sample1_10_1_0.json')
print(dt.data.shape, len(dt.labels), dt.meta[0])

# load data into Format1 
loader = Format1_loader(data={
    'data': [[1,2,3], [4,5,6]],
    'labels': [0, 1],
    'meta': [{'param1': 1}, {'param2': 2}]
})

# Accessing properties
print(isinstance(loader.data, np.ndarray))  
print(loader.meta)

(10, 101) 10 {'noise': {'hash': '93b8f7d8d5f5bc245bd519ddbd548f3e', 'params': {'general': {'length': 101, 'type': 'Noise_generator', 'generator_name': 'normal_noise'}, 'gen_params': {'noise_pct': 0.0}}}, 'trend': {'hash': '5963358791788bc1521d4beb0c5f4964', 'params': {'general': {'length': 101, 'type': 'TS_generator', 'generator_name': 'exponential_trend'}, 'gen_params': {'alpha': 0.03, 'length': 101}}}}
True
[{'param1': 1}, {'param2': 2}]


In [4]:
dt.data[0][:10]

array([1.        , 1.03045453, 1.06183655, 1.09417428, 1.12749685,
       1.16183424, 1.19721736, 1.23367806, 1.27124915, 1.30996445])

## ts_classes

In [2]:
from patrec.utils.ts_classes import *

In [3]:
data = [1.        , 1.03045453, 1.06183655, 1.09417428, 1.12749685,
       1.16183424, 1.19721736, 1.23367806, 1.27124915, 1.30996445]

comp1 = TS_component(data, component_type=TS_component_type.SIGNAL, name='segment_1')
comp2 = TS_component(data, component_type=TS_component_type.SIGNAL, name='segment_2')

block1 = TS_additive_block()
block1.add_component(comp1)

block2 = TS_additive_block()
block2.add_component(comp2)

print(block1.get_metadata(), block1.components)

# Connect blocks
connector = Block_connector()
connector.add_block(block1)
connector.add_block(block2)

print(connector._blocks)

# Get complete series with smooth transitions
full_series = connector.build_series(smooth_transitions=True)

# Get just the trend components
trend_series = connector.get_component_series(TS_component_type.SIGNAL)
print(trend_series)

# Rearrange blocks
connector.rearrange_blocks([1, 0])

print(connector._transition_cache, connector.get_metadata())

{'components': {'SIGNAL': {'hash': '931ec57b819cd9bf2c40baead8c4d562', 'params': {}, 'name': 'segment_1'}}, 'block_metadata': {}} {<TS_component_type.SIGNAL: 1>: <patrec.utils.ts_classes.TS_component object at 0x7d4b397db0e0>}
[<patrec.utils.ts_classes.TS_additive_block object at 0x7d4d7c14fda0>, <patrec.utils.ts_classes.TS_additive_block object at 0x7d4d7c14c830>]
[1.         1.03045453 1.06183655 1.09417428 1.12749685 1.16183424
 1.19721736 1.23367806 1.27124915 1.30996445 1.         1.03045453
 1.06183655 1.09417428 1.12749685 1.16183424 1.19721736 1.23367806
 1.27124915 1.30996445]
{} {'blocks': [{'components': {'SIGNAL': {'hash': '931ec57b819cd9bf2c40baead8c4d562', 'params': {}, 'name': 'segment_2'}}, 'block_metadata': {}}, {'components': {'SIGNAL': {'hash': '931ec57b819cd9bf2c40baead8c4d562', 'params': {}, 'name': 'segment_1'}}, 'block_metadata': {}}], 'total_length': 20}
